# **Summariser for text files**

Installing modules and importing files:

In [33]:
!pip install networkx
!pip install scipy==1.8.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
import numpy as np
import pandas as pd
import regex as re
import networkx as nx
import nltk
from nltk.cluster.util import cosine_distance
import networkx as nx
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import files
files.upload()

Saving file_name.txt to file_name.txt


{'file_name.txt': b'"But I must explain to you how all this mistaken idea of denouncing pleasure and praising pain was born and I will give you a \ncomplete account of the system, and expound the actual teachings of the great explorer of the truth, the master-builder of \nhuman happiness. No one rejects, dislikes, or avoids pleasure itself, because it is pleasure, but because those who do not know \nhow to pursue pleasure rationally encounter consequences that are extremely painful. Nor again is there anyone who loves or \npursues or desires to obtain pain of itself, because it is pain, but because occasionally circumstances occur in which toil and \npain can procure him some great pleasure. To take a trivial example, which of us ever undertakes laborious physical exercise, \nexcept to obtain some advantage from it? But who has any right to find fault with a man who chooses to enjoy a pleasure that \nhas no annoying consequences, or one who avoids a pain that produces no resultant plea

We use the pre-trained Wikipedia 2014 + Gigaword 5 GloVe vectors available here. 

Word embeddings are a word's vector representation. Our sentences' vectors will be made using these word embeddings.

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2022-12-31 10:47:05--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-12-31 10:47:05--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-12-31 10:47:05--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [35]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

Replacing punctuations, numbers and special characters with a " ".

In [36]:
reading_file = open("eng.txt", "r")

new_file_content = ""
for line in reading_file:
    stripped_line = line.strip()
    clean_sentences = stripped_line.replace("[^a-zA-Z]", " ")
    new_file_content += clean_sentences +"\n"
reading_file.close()

writing_file = open("file_name.txt", "w")
writing_file.write(new_file_content)
writing_file.close()

In [37]:
# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [38]:
stop_words = stopwords.words('english')

Removing Stopwords:

Stopwords are commonly used words such as verbs (like is, am), prepositions (in, etc) or articles. They're words that do not hold much weight in our analysis of sentences, and hence, are better removed.

In [39]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [40]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [41]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [42]:
file = open("file_name.txt", "r")

In [43]:
sentences = []
for s in file:
  sentences.append(sent_tokenize(s))

In [44]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [45]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

We convert our similarity matrix calculated above into a graph. We apply the PageRank algorithm below:

In [46]:
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [47]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [48]:
# Extract top 10 sentences as the summary
for i in range(10):
  print(ranked_sentences[i][1])

['how to pursue pleasure rationally encounter consequences that are extremely painful.', 'Nor again is there anyone who loves or']
['complete account of the system, and expound the actual teachings of the great explorer of the truth, the master-builder of']
['have to be repudiated and annoyances accepted.', 'The wise man therefore always holds in these matters to this principle of']
['and pain.', 'These cases are perfectly simple and easy to distinguish.', 'In a free hour, when our power of choice is untrammelled and']
['blame belongs to those who fail in their duty through weakness of will, which is the same as saying through shrinking from toil']
[]
['selection: he rejects pleasures to secure other greater pleasures, or else he endures pains to avoid worse pains."']
['pursues or desires to obtain pain of itself, because it is pain, but because occasionally circumstances occur in which toil and']
['human happiness.', 'No one rejects, dislikes, or avoids pleasure itself, because it is 